In [3]:
!which python

/Users/priya/Downloads/UC Davis/Assignments/venv/bin/python


In [ ]:
pip3 install -r requirements.txt

anyio==4.8.0
appnope==0.1.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.4
attrs==25.1.0
babel==2.17.0
beautifulsoup4==4.13.3
bleach==6.2.0
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
comm==0.2.2
debugpy==1.8.13
decorator==5.2.1
defusedxml==0.7.1
exceptiongroup==1.2.2
executing==2.2.0
fastjsonschema==2.21.1
fqdn==1.5.1
h11==0.14.0
httpcore==1.0.7
httpx==0.28.1
idna==3.10
importlib-metadata==8.6.1
ipykernel==6.29.5
ipython==8.18.1
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.2
jinja2==3.1.5
json5==0.10.0
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter==1.1.1
jupyter-client==8.6.3
jupyter-console==6.6.3
jupyter-core==5.7.2
jupyter-events==0.12.0
jupyter-lsp==2.2.5
jupyter-server==2.15.0
jupyter-server-terminals==0.5.3
jupyterlab==4.3.5
jupyterlab-pygments==0.3.0
jupyterlab-server==2.27.3
jupyterlab-widgets==3.0.13
MarkupSafe==3.0.2
matplotlib-inline==0.1.7
mistune==3.1.2
nbclient==0.10.

In [10]:
import undetected_chromedriver as uc
import chromedriver_py  # this imports the chromedriver path
import time
import random
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.6943.142 Safari/537.36")

# Get ChromeDriver path from chromedriver_py
driver_path = chromedriver_py.binary_path

# Set up undetected ChromeDriver with the ChromeDriver path
driver = uc.Chrome(driver_executable_path=driver_path, options=options)

# Open the DoorDash search page with the search query
search_input = input("Enter the store type or keyword to search on DoorDash (e.g., 'sushi'): ")
url = f"https://www.doordash.com/search/store/{search_input}?event_type=search"
driver.get(url)

# Add some delay before interacting with the page
time.sleep(random.uniform(2, 5))

# Execute JavaScript to hide webdriver detection
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Give time for the page to load
time.sleep(random.uniform(2, 5))

# Close browser
input("Press Enter to close the browser...")
driver.quit()
